## Importing the necessary modules

In [1]:
from json import loads
from keras import Sequential
from keras.layers import Dense, Dropout, LeakyReLU, Softmax
from keras.optimizers import Adam
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
from numpy import array
from random import choice, shuffle
from string import punctuation
'''
nltk.download("punkt")
nltk.download("wordnet")
nltk.download('omw-1.4')
'''


'\nnltk.download("punkt")\nnltk.download("wordnet")\nnltk.download(\'omw-1.4\')\n'

## Loading the dataset

In [2]:
data = loads(open("dataset/intents.json").read())


## Using NLP to clean the data

In [3]:
# initializing lemmatizer to get stem of words
lemmatizer = WordNetLemmatizer()

# Each list to create
words = []
classes = []
doc_X = []
doc_y = []

'''Looping through all the intents and tokenizing each patterns
and appending tokens to words, the patterns 
and the associated tag to their associated list'''
for intent in data["intents"]:
    for pattern in intent["patterns"]:
        tokens = word_tokenize(pattern)
        words.extend(tokens)
        doc_X.append(pattern)
        doc_y.append(intent["tag"])
    # add the tag to the classes if it's not there already
    if intent["tag"] not in classes:
        classes.append(intent["tag"])

'''lemmatizing all the words in the vocab 
and converting them to lowercase
if the words don't appear in punctuation'''
words = [lemmatizer.lemmatize(word.lower()) for word in words if word not in punctuation]

'''sorting the vocab and classes in alphabetical order 
and taking the set to ensure no duplicates occur'''
words = sorted(set(words))
classes = sorted(set(classes))


## Creating training and test sets

In [4]:
# list for training data
training = []
out_empty = [0] * len(classes)

# creating the bag of words model
for idx, doc in enumerate(doc_X):
    bow = []
    text = lemmatizer.lemmatize(doc.lower())
    for word in words:
        bow.append(1) if word in text else bow.append(0)
    # marking the index of class that the current pattern is associated to
    output_row = list(out_empty)
    output_row[classes.index(doc_y[idx])] = 1
    # adding the one hot encoded BoW and associated classes to training
    training.append([bow, output_row])

# shuffling the data and convert it to an array
shuffle(training)
training = array(training, dtype=object)

# splitting the features and target labels
train_X = array(list(training[:, 0]))
train_y = array(list(training[:, 1]))


# Building deep learning model

In [5]:
# defining some parameters
input_shape = (len(train_X[0]),)
output_shape = len(train_y[0])

# defining the model
model = Sequential()
model.add(Dense(128, input_shape=input_shape, activation=LeakyReLU(alpha=0.3)))
model.add(Dropout(0.25))
model.add(Dense(64, activation=LeakyReLU(alpha=0.3)))
model.add(Dropout(0.25))
model.add(Dense(output_shape, activation=Softmax()))

# compiling the model
adam = Adam(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=["accuracy"])
print(model.summary())


Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               14848     
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 23)                1495      
                                                                 
Total params: 24,599
Trainable params: 24,599
Non-trainable params: 0
__________________________________________

2022-11-02 21:57:39.891888: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-11-02 21:57:39.892372: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


## Training the model

In [6]:
model.fit(x=train_X, y=train_y, epochs=200)


Epoch 1/200


2022-11-02 21:57:40.206236: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-11-02 21:57:40.693346: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


4/4 [==============================] - 1s 159ms/step - loss: 3.0768 - accuracy: 0.0721
Epoch 2/200
4/4 [==============================] - 0s 13ms/step - loss: 2.5879 - accuracy: 0.4955
Epoch 3/200
4/4 [==============================] - 0s 16ms/step - loss: 2.0079 - accuracy: 0.6216
Epoch 4/200
4/4 [==============================] - 0s 14ms/step - loss: 1.3536 - accuracy: 0.7207
Epoch 5/200
4/4 [==============================] - 0s 12ms/step - loss: 0.8566 - accuracy: 0.8378
Epoch 6/200
4/4 [==============================] - 0s 13ms/step - loss: 0.5201 - accuracy: 0.9099
Epoch 7/200
4/4 [==============================] - 0s 14ms/step - loss: 0.3197 - accuracy: 0.9279
Epoch 8/200
4/4 [==============================] - 0s 14ms/step - loss: 0.2219 - accuracy: 0.9279
Epoch 9/200
4/4 [==============================] - 0s 14ms/step - loss: 0.1415 - accuracy: 0.9730
Epoch 10/200
4/4 [==============================] - 0s 12ms/step - loss: 0.1047 - accuracy: 1.0000
Epoch 11/200
4/4 [============

# Functions for getting results

In [7]:
# defining a function to clean the text
def clean_text(text):
    tokens = word_tokenize(text)
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    return tokens

# defining a function to get the bag of words
def bag_of_words(text, vocab):
    tokens = clean_text(text)
    bow = [0] * len(vocab)
    for w in tokens:
        for idx, word in enumerate(vocab):
            if word == w:
                bow[idx] = 1
    return array(bow)

# defining a function to predict the class
def pred_class(text, vocab, labels):
    bow = bag_of_words(text, vocab)
    result = model.predict(array([bow]))[0]
    thresh = 0.2
    y_pred = [[idx, res] for idx, res in enumerate(result) if res > thresh]
    y_pred.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in y_pred:
        return_list.append(labels[r[0]])
    return return_list

# defining a function to get the response
def get_response(intents_list, intents_json):
    tag = intents_list[0]
    list_of_intents = intents_json["intents"]
    for i in list_of_intents:
        if i["tag"] == tag:
            result = choice(i["responses"])
            break
    return result



## Testing the model

In [8]:
from datetime import datetime
now = datetime.now()
print("Anuradha is ready to chat! (type 'exit' to quit)")
while True:
    try:
        message = input("You : ")
        if message == 'exit':
            TEXT = "Bye! take care"
            print("Anuradha :", TEXT)
            break
        intents = pred_class(message, words, classes)
        result = get_response(intents, data)
        if result == "date":
            TEXT = now.strftime(r'%d/%m/%Y')
            print("Anuradha :", TEXT)
        elif result == "time":
            TEXT = now.strftime(r'%H:%M:%S')
            print("Anuradha :", TEXT)
        else:
            print("Anuradha :", result)
    except ValueError:
        TEXT = "Sorry, I didn't get that"
        print("Anuradha :", TEXT)


Anuradha is ready to chat! (type 'exit' to quit)
1/1 [==============================] - 0s 104ms/step


2022-11-02 21:58:01.851290: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


Anuradha : Greetings!
1/1 [==============================] - 0s 18ms/step
Anuradha : Hi there, how can I help?
1/1 [==============================] - 0s 26ms/step
Anuradha : Hi, I'm Anuradha, a Deep-Learning chatbot
1/1 [==============================] - 0s 18ms/step
Anuradha : 21:57:53
1/1 [==============================] - 0s 17ms/step
Anuradha : 02/11/2022
1/1 [==============================] - 0s 18ms/step
Anuradha : The world tongue-twister champion just got arrested. I hear they're gonna give him a really tough sentence.
1/1 [==============================] - 0s 17ms/step
Anuradha : Haha, I am glad you found it funny
1/1 [==============================] - 0s 17ms/step
Anuradha : What two things can you never eat for breakfast?.....Lunch and Dinner!
1/1 [==============================] - 0s 18ms/step
Anuradha : Thanks for the laugh
1/1 [==============================] - 0s 17ms/step
Anuradha : Thank you so much!
1/1 [==============================] - 0s 17ms/step
Anuradha : I am s